<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-tools/blob/main/notebooks/Eye_ai_get_cropped_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Cropping

This notebook applied a pre-trained model to a dataset specified in the configuration file and uploads the cropped images' bounding box to the catalog.

In [ ]:
# Prerequisites to configure colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install deriva
    !pip install bdbag
    !pip install --upgrade --force pydantic
    !pip install git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml
    !pip install setuptools_git_versioning


In [ ]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [ ]:
import json
import os
import eye_ai
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
print(f"Using eye_ai version: {eye_ai.__version__}")
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
# @title login to DERIVA via Globus Auth

DEBUG_MODE = True #@param ["False", "True"] {type:"raw"}
catalog_id = "eye-ai" #@param
DEFAULT_SERVER = 'dev.eye-ai.org' if DEBUG_MODE else 'www.eye-ai.org'

!deriva-globus-auth-utils login --no-browser --host {DEFAULT_SERVER}

In [ ]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-9F8W"      # Configuration file for this run.  Needs to be changed for each execution.

In [ ]:
EA = EyeAI(hostname = DEFAULT_SERVER, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)


In [ ]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
input_dataset = configuration_records.bag_paths[0] # Assumes that the configuration file only specifies one dataset.
configuration_records.model_dump()

In [ ]:
# @title Data Preprocessing (Filtering Image.csv for just Field_2 Images)
field2_csv_path = EA.filter_angle_2(configuration_records.bag_paths[0])
field2_csv_path

In [ ]:
EA.execution_assets_path

In [ ]:
# @title Execute Proecss algorithm (Cropping)
from eye_ai_ml.glaucoma.optic_disk_crop import preprocess_and_crop
with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  preprocess_and_crop(
      str(configuration_records.bag_paths[0])+"/data/assets/Image/",
      field2_csv_path,
      str(EA.working_dir)+'/output.csv',
      'template.jpg',
      str(EA.execution_assets_path),
      configuration_records.assets_paths[0],
      configuration_records.vocabs['Annotation_Type'][0].rid,
      configuration_records.vocabs['Annotation_Type'][0].name,
      False
      )

In [ ]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)
